In [11]:
import re


database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

# ----------- Exemplos errados ------------
#exemplo = "join contas on usuario.idUsuario = contas.Usuario_idUsuario"
#exemplo = "selec * from usuario"
exemplo = "selec * fro usuario"
exemplo = "select * froM usuario wher manp"
exemplo = "select * froM usuario joi contas on mano where mano > 3"

# ----------- Exemplos certos ------------

#exemplo = "Select idusuario, nome, datanascimento, descricao, saldoinicial, UF, Descrição from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta where saldoinicial < 3000 and uf = 'ce' and Descrição <> 'Conta Corrente' and idusuario > 3"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario where saldoinicial >=235 and uf ='ce' and cep <> '62930000' and name='alex';"
exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join Categoria on a = usuario.b"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario"
exemplo = "Select Nome, DataNascimento from Usuario join tipoconta on tipoconta.idtipoconta = usuario.idusuario where Logradouro >=  235"
exemplo = "Select Usuario.Nome, Logradouro from Usuario join contas on idcontas = usuario.idusuario and idcontas <> idusuario join movimentacao on descricao = idmovimentacao"
def regex(database,string):
    string = string.lower()
    todas_colecoes = []

    #selecionando o select from
    pattern_select_from = r"^\bSelect\b\s(.*?)\s\bFrom\b\s(.*?)(($|join|where)(.*?)$)"
    matches = re.findall(pattern_select_from, string, re.IGNORECASE)
    matches_select_from = matches
    
    if len(matches_select_from) == 0:
        return False
    valores_chamados = matches_select_from[0][0].strip()
    colecao_chamada = matches_select_from[0][1].strip()
    outros_valores = matches_select_from[0][2].strip()

    todas_colecoes.append(colecao_chamada)

    if not colecao_chamada in database:
        return False
    
    
    pattern_select_from = r"(?:(.*?)\bwhere\b(.*?))$"
    matches = re.findall(pattern_select_from, outros_valores, re.IGNORECASE)
    matches_where = matches
    

    if len(matches_where) > 0:
        leitura_join_on = matches_where[0][0]
    else:
        leitura_join_on = outros_valores

    if leitura_join_on.count("(") != leitura_join_on.count(")"):
        return False
    leitura_join_on = leitura_join_on.replace("(","")
    leitura_join_on = leitura_join_on.replace(")","")

    #checagem e tratamento do join on
    pattern_join_on = r"(?:\bjoin\b(.*?)(?:\bon\b|$)(.*?)(?=\bJoin\b|$))"
    matches = list(filter(lambda x: x != ("", "",""), re.findall(pattern_join_on, leitura_join_on, re.IGNORECASE)))
    matches_join_on = matches
    for i in matches_join_on:
        colecao = i[0].strip()
        if colecao not in database:
            return False
        todas_colecoes.append(colecao)
        caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,i[1].strip())))


        for valor in caracteres:

            valor = valor.strip()
            if '.' in valor:
                
                colecao_valor = valor.split(".")

                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            
            else:
                colecao_inexistente = True
                for colecao in todas_colecoes:

                    if valor in database[colecao]:
                        colecao_inexistente = False
                
                if colecao_inexistente:
                    return False
                
    if len(matches_where) > 0:
        catacteres_where = matches_where[0][1].split("and")
        for i in catacteres_where:

            if i == catacteres_where[-1]:
                caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>|\bin\b|\bnot in\b' ,i.strip())))
            else:
                caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,i.strip())))

            valor = caracteres[0].strip()
            if '.' in valor:
                
                colecao_valor = valor.split(".")
                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            
            else:
                colecao_inexistente = True
                for colecao in todas_colecoes:

                    if valor in database[colecao]:
                        colecao_inexistente = False
                
                if colecao_inexistente:
                    return False
                
    valores_chamados = valores_chamados.split(",")

    if len(valores_chamados) == 1 and valores_chamados[0] == "*":
        return string
    else:
        for valor in valores_chamados:

            if '.' in valor:
                colecao_valor = valor.split(".")
                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            else:
                valor_inexistente = True
                for i in todas_colecoes:
                    if valor in database[i]:
                        valor_inexistente = False
                
                if valor_inexistente:
                    return False


        return string



database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

#exemplo = "select * cont1as.idconta from contas join tipoconta on tipoconta.idtipoconta = contas.idconta"
text = "select * from contas join tipoconta on usuario.idconta = contas.idconta"
text = "select * from contas join tipoconta on usuario.idconta = contas.idconta"
#text = "select * from contas join tipoconta on contas.idconta = tipoconta.idconta"
#text = "select * from contas join tipoconta on contas.idconta = tipoconta.idconta"
text = "select * from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta join Movimentacao on movimentacao.contas_idconta = contas.idconta where uf = 'ce' and saldoInicial not in casa"
#text = "select * from tipoconta join categoria on tipoconta.descrição = categoria.idcategoria where"
text = "select * from tipomovimento where idtipomovimento not in (1,2,3,4,5) and idtipomovimento = 1"
print(regex(database,text))


False


Errada:

In [ ]:
import re


database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

# ----------- Exemplos errados ------------
#exemplo = "join contas on usuario.idUsuario = contas.Usuario_idUsuario"
#exemplo = "selec * from usuario"
exemplo = "selec * fro usuario"
exemplo = "select * froM usuario wher manp"
exemplo = "select * froM usuario joi contas on mano where mano > 3"

# ----------- Exemplos certos ------------

#exemplo = "Select idusuario, nome, datanascimento, descricao, saldoinicial, UF, Descrição from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta where saldoinicial < 3000 and uf = 'ce' and Descrição <> 'Conta Corrente' and idusuario > 3"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario where saldoinicial >=235 and uf ='ce' and cep <> '62930000' and name='alex';"
exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join Categoria on a = usuario.b"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario"
exemplo = "Select Nome, DataNascimento from Usuario join tipoconta on tipoconta.idtipoconta = usuario.idusuario where Logradouro >=  235"
exemplo = "Select Usuario.Nome, Logradouro from Usuario join contas on idcontas = usuario.idusuario and idcontas <> idusuario join movimentacao on descricao = idmovimentacao"
def regex(database,string):
    string = string.lower()
    colecoes_chamadas = []

    #selecionando o select from
    pattern_select_from = r"^\bSelect\b\s(.*?)\s\bFrom\b\s(.*?)(($|join|where)(.*?)$)"
    matches = re.findall(pattern_select_from, string, re.IGNORECASE)
    matches_select_from = matches

    if len(matches) == 0:
        return False
    else:

        colecoes_chamadas.append(matches_select_from[0][1].strip())
        coluna = matches_select_from[0][1].strip()

        if coluna not in database:
            return False
        
        elementos = matches_select_from[0][0].split(",")

            
        
    #checagem do where
    m1 = matches_select_from[0][2]
    pattern_select_from = r"(?:(.*?)\bwhere\b(.*?))$"
    matches = re.findall(pattern_select_from, m1, re.IGNORECASE)
    matches_where = matches

    if len(matches_where) == 0:
        m1 = matches_select_from[0][2]
    else:
        m1 = matches_where[0][0]
        valor_where = matches_where[0][1]


    #checagem e tratamento do join on
    pattern_join_on = r"(?:\bjoin\b(.*?)(?:\bon\b|$)(.*?)(?=\bJoin\b|$))"
    matches = list(filter(lambda x: x != ("", "",""), re.findall(pattern_join_on, m1, re.IGNORECASE)))
    matches_join_on = matches

    for i in matches_join_on:
        print(i)
        caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>|\band\b' ,i[1].strip())))


        if i[0].strip() in database:
            colecoes_chamadas.append(i[0].strip())
            colecao_inexistente = True
            for i in colecoes_chamadas:

                for j in caracteres:
                    if '.' in j:

                        if j.split('.')[0].strip() not in colecoes_chamadas:
                            print(j)
                            return False
                        j = j.split('.')[1]
                    if j.strip() in database[i]:
                        colecao_inexistente = False
            
            if colecao_inexistente == True:
                return False
            
        else:
            return False


    #tratamento do where
    if len(matches_where) > 0:
        print(matches_where)
        if len(matches_where[0][1]) != 0:
            valor_where = matches_where[0][1]
            #checagem do and
            pattern_where_and = r"(.*?)(?:\bAND\b\s)(.*?)(?=\s\bAND\b|$)"
            matches = list(filter(lambda x: x != ("", "",""), re.findall(pattern_where_and, valor_where, re.IGNORECASE)))
            if len(matches) > 0:
                for i in matches:
                    print(i)
                    for j in i:  
                        if len(j.strip()) != 0:
                            caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,j.strip())))
                            if len(caracteres) != 2:
                                return False
                            else:
                                colecao_inexistente = True
                                for i in colecoes_chamadas:
                                    tabela = i.strip()
                                    if caracteres[0].strip() in database[tabela]:
                                        colecao_inexistente = False
                                if colecao_inexistente == True:
                                    return False
            else:
                caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,valor_where.strip())))
                if len(caracteres) != 2:
                    return False
                else:
                    colecao_inexistente = True
                    for i in colecoes_chamadas:
                        tabela = i.strip()
                        if '.' in caracteres[0]:
                            caracteres[0] = caracteres[0].split('.')[1]
                        if caracteres[0].strip() in database[tabela]:
                            colecao_inexistente = False

                    if colecao_inexistente == True:
                        return False

    #editar aqui
    for i in elementos:
        print(i)
        if i == "*" and len(elementos) == 1:
            print("valido")
        elif '.' in i:
            elemento_com_ponto = i.strip().split(".")
            colecao_inexistente = True
            if elemento_com_ponto[0].strip() not in colecoes_chamadas:
                return False
            for j in colecoes_chamadas:
                if elemento_com_ponto[1] in database[j]:
                    colecao_inexistente = False
            
            if colecao_inexistente:
                return False
                







    return string

database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

#select * from contas join tipoconta on contas.TipoConta_idTipoConta= tipoconta.idTipoConta
#select * from movimentacao join tipomovimento on movimentacao.tipomovimento_idtipomovimento = tipomovimento.idtipomovimento
exemplo = "select * from movimentacao join tipomovimento on movimentacao.tipomovimento_idtipomovimento = tipomovimento.idtipomovimento"
regex(database,exemplo)


In [150]:
database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

#select * from contas join tipoconta on contas.TipoConta_idTipoConta= tipoconta.idTipoConta
#select * from movimentacao join tipomovimento on movimentacao.tipomovimento_idtipomovimento = tipomovimento.idtipomovimento
# * entre select e from
#where escrito errado
exemplo = "select * from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta join Movimentacao on movimentacao.contas_idconta = contas.idconta wh2ere uf = 'ce' and saldoInicial = 200"
regex(database,exemplo)

(' contas ', ' usuario.idusuario = contas.usuario_idusuario ')
(' tipoconta ', ' tipoconta.idtipoconta = contas.tipoconta_idtipoconta ')
(' movimentacao ', " movimentacao.contas_idconta = contas.idconta wh2ere uf = 'ce' and saldoinicial = 200")
*
valido


"select * from usuario join contas on usuario.idusuario = contas.usuario_idusuario join tipoconta on tipoconta.idtipoconta = contas.tipoconta_idtipoconta join movimentacao on movimentacao.contas_idconta = contas.idconta wh2ere uf = 'ce' and saldoinicial = 200"

In [34]:
import re


database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

# ----------- Exemplos errados ------------
#exemplo = "join contas on usuario.idUsuario = contas.Usuario_idUsuario"
#exemplo = "selec * from usuario"
exemplo = "selec * fro usuario"
exemplo = "select * froM usuario wher manp"
exemplo = "select * froM usuario joi contas on mano where mano > 3"

# ----------- Exemplos certos ------------

#exemplo = "Select idusuario, nome, datanascimento, descricao, saldoinicial, UF, Descrição from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta where saldoinicial < 3000 and uf = 'ce' and Descrição <> 'Conta Corrente' and idusuario > 3"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario where saldoinicial >=235 and uf ='ce' and cep <> '62930000' and name='alex';"
exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join Categoria on a = usuario.b"
#exemplo = "Select nome, datanascimento, descricao, saldoinicial from usuario"
exemplo = "Select Nome, DataNascimento from Usuario join tipoconta on tipoconta.idtipoconta = usuario.idusuario where Logradouro >=  235"
exemplo = "Select Usuario.Nome, Logradouro from Usuario join contas on idcontas = usuario.idusuario and idcontas <> idusuario join movimentacao on descricao = idmovimentacao"
def regex(database,string):
    string = string.lower()
    todas_colecoes = []

    #selecionando o select from
    pattern_select_from = r"^\bSelect\b\s(.*?)\s\bFrom\b\s(.*?)(($|join|where)(.*?)$)"
    matches = re.findall(pattern_select_from, string, re.IGNORECASE)
    matches_select_from = matches
    
    if len(matches_select_from) == 0:
        return False
    valores_chamados = matches_select_from[0][0].strip()
    colecao_chamada = matches_select_from[0][1].strip()
    outros_valores = matches_select_from[0][2].strip()

    todas_colecoes.append(colecao_chamada)

    if not colecao_chamada in database:
        return False
    
    
    pattern_select_from = r"(?:(.*?)\bwhere\b(.*?))$"
    matches = re.findall(pattern_select_from, outros_valores, re.IGNORECASE)
    matches_where = matches


    if len(matches_where) > 0:
        leitura_join_on = matches_where[0][0]
    else:
        leitura_join_on = outros_valores

    #checagem e tratamento do join on
    pattern_join_on = r"(?:\bjoin\b(.*?)(?:\bon\b|$)(.*?)(?=\bJoin\b|$))"
    matches = list(filter(lambda x: x != ("", "",""), re.findall(pattern_join_on, leitura_join_on, re.IGNORECASE)))
    matches_join_on = matches
    for i in matches_join_on:
        colecao = i[0].strip()
        if colecao not in database:
            return False
        todas_colecoes.append(colecao)
        caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,i[1].strip())))


        for valor in caracteres:

            valor = valor.strip()
            if '.' in valor:
                
                colecao_valor = valor.split(".")

                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            
            else:
                colecao_inexistente = True
                for colecao in todas_colecoes:

                    if valor in database[colecao]:
                        colecao_inexistente = False
                
                if colecao_inexistente:
                    return False
                
    if len(matches_where) > 0:
        for i in matches_where[0][1].split("and"):
            caracteres = list(filter(bool,re.split(r'<=|>=|=|<|>|<>' ,i.strip())))

            valor = caracteres[0].strip()
            if '.' in valor:
                
                colecao_valor = valor.split(".")
                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            
            else:
                colecao_inexistente = True
                for colecao in todas_colecoes:

                    if valor in database[colecao]:
                        colecao_inexistente = False
                
                if colecao_inexistente:
                    return False
                
    valores_chamados = valores_chamados.split(",")

    if len(valores_chamados) == 1 and valores_chamados[0] == "*":
        return string
    else:
        for valor in valores_chamados:

            if '.' in valor:
                
                colecao_valor = valor.split(".")
                try:
                    if not colecao_valor[1].strip() in database[colecao_valor[0].strip()]:
                        return False
                except:
                    return False
            else:
                valor_inexistente = True
                for i in todas_colecoes:
                    if valor in database[i]:
                        valor_inexistente = False
                
                if valor_inexistente:
                    return False


        return string



database = {
    "usuario": ["idusuario", "nome", "logradouro", "numero", "bairro", "cep", "uf", "datanascimento"],
    "contas": ["idconta", "descricao", "tipoconta_idtipoconta", "usuario_idusuario", "saldoinicial"],
    "movimentacao": ["idmovimentacao", "datamovimentacao", "descricao", "tipomovimento_idtipomovimento", "categoria_idcategoria", "contas_idconta", "valor"],
    "tipoconta": ["idtipoconta", "descrição"],
    "tipomovimento": ["idtipomovimento", "descmovimentacao"],
    "categoria": ["idcategoria", "desccategoria"]
}

#exemplo = "select * cont1as.idconta from contas join tipoconta on tipoconta.idtipoconta = contas.idconta"
text = "select * from contas join tipoconta on usuario.idconta = contas.idconta"
text = "select * from contas join tipoconta on usuario.idconta = contas.idconta"
#text = "select * from contas join tipoconta on contas.idconta = tipoconta.idconta"
#text = "select * from contas join tipoconta on contas.idconta = tipoconta.idconta"
text = "select * from usuario join contas on usuario.idUsuario = contas.Usuario_idUsuario join tipoconta on tipoconta.idTipoConta = contas.TipoConta_idTipoConta join Movimentacao on movimentacao.contas_idconta = contas.idconta where uf = 'ce' and saldoInicial = 200"
print(regex(database,text))


IndentationError: expected an indented block after 'for' statement on line 37 (935062535.py, line 38)

In [19]:
a = "Usuario.Nome".strip().split(".")
if a[1] in database[a[0]]:
    print("aa")

aa


In [72]:
if 'Logradouro' in database['Usuario']:
    print("a")

a


In [55]:
import re

pattern = r"^Select.*\bFrom.*(\bJoin.*\bOn.*)?"
#string1 = "Select * From Table1"
string2 = "Select tabela1.coluna1, tabela2.coluna2 From tabela1 Join tabela2 On tabela1.chave = tabela2.chave"
#string3 = "Select * From Table1 Join Table2"




In [ ]:
0%2

0

In [31]:
contents = regex(exemplo)
print(contents)
# Output: ['b', 'a(b)c']

TypeError: regex() missing 1 required positional argument: 'string'

In [ ]:


# String de exemplo
string = extract_parentheses_order("Essa é uma ((string) de (exemplo)) com (parênteses).")

string

[['(', '(', ')', '(', ')', ')'], ['(', ')']]

In [24]:
def remover_caracteres(string_inicial):
    caracteres_para_remover = "\n,;"
    for caracter in caracteres_para_remover:
        string_inicial = string_inicial.replace(caracter, " ")

    return string_inicial


In [ ]:
def modelo_parser(consulta_sql):

    #elementos do parse em maiusculo
    Parse = ["SELECT", "FROM", "WHERE", "JOIN", "ON", "=", ">", "<", "<=", ">=", "<>", "AND", "IN", "NOT", "(" , ")" ]
    itens = remover_caracteres(consulta_sql).split() #remover caracteres "lixo"

    #divisao dos elementos
    colunas_e_tabelas = [palavra for palavra in itens if palavra not in Parse]
    expressoes_no_elemento = [palavra for palavra in itens if palavra in Parse]

    #ver os elementos
    for i in itens:
        if i in colunas_e_tabelas:
            print("Expressao:{}".format(i))
        else:
            print("Item:{}".format(i))



In [ ]:
modelo_parser(exemplo)

TypeError: 'str' object is not callable